In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Email Parsing Experimentation\n",
    "\n",
    "This notebook explores different approaches to parsing email content using LangChain."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "import json\n",
    "from dotenv import load_dotenv\n",
    "\n",
    "# Add parent directory to path\n",
    "sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))\n",
    "\n",
    "# Load environment variables\n",
    "load_dotenv()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from langchain.llms import OpenAI\n",
    "from langchain.prompts import PromptTemplate\n",
    "from langchain.chains import LLMChain\n",
    "\n",
    "# Initialize LLM\n",
    "llm = OpenAI(temperature=0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Test Email Templates"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example emails\n",
    "test_emails = [\n",
    "    \"\"\"\n",
    "    Subject: Feedback on Widget X\n",
    "    \n",
    "    Hello,\n",
    "    \n",
    "    I recently purchased your Widget X and I'm very impressed with the quality.\n",
    "    It's exactly what I needed for my project. The build quality is excellent\n",
    "    and it works perfectly.\n",
    "    \n",
    "    One small suggestion: it would be great if it came with a storage case.\n",
    "    \n",
    "    Thanks,\n",
    "    John Smith\n",
    "    \"\"\",\n",
    "    \n",
    "    \"\"\"\n",
    "    Subject: Problem with my order #12345\n",
    "    \n",
    "    Hi Support Team,\n",
    "    \n",
    "    I ordered Product Y on Monday and it still hasn't arrived. The tracking\n",
    "    number isn't working either. Can you please check what's happening with\n",
    "    my order? I need it urgently for an upcoming project.\n",
    "    \n",
    "    Regards,\n",
    "    Jane Doe\n",
    "    \"\"\",\n",
    "    \n",
    "    \"\"\"\n",
    "    Subject: Request for information\n",
    "    \n",
    "    Hello,\n",
    "    \n",
    "    I'm interested in your Widget Z but I couldn't find information about\n",
    "    whether it's compatible with XYZ systems. Could you please let me know?\n",
    "    \n",
    "    Also, do you offer bulk discounts for purchases over 20 units?\n",
    "    \n",
    "    Thanks,\n",
    "    Michael Johnson\n",
    "    \"\"\"\n",
    "]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Test Different Prompt Templates"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Basic prompt template\n",
    "basic_prompt = PromptTemplate(\n",
    "    input_variables=[\"email\"],\n",
    "    template=\"\"\"\n",
    "    Extract the following information from this email:\n",
    "    1. Customer name\n",
    "    2. Product mentioned\n",
    "    3. Sentiment (positive, negative, neutral)\n",
    "    4. Main issue or request\n",
    "    \n",
    "    Email:\n",
    "    {email}\n",
    "    \n",
    "    Format as JSON with keys: customer_name, product, sentiment, main_issue\n",
    "    \"\"\"\n",
    ")\n",
    "\n",
    "# Enhanced prompt template\n",
    "enhanced_prompt = PromptTemplate(\n",
    "    input_variables=[\"email\"],\n",
    "    template=\"\"\"\n",
    "    Extract the following information from this email:\n",
    "    1. Customer name (if available)\n",
    "    2. Product mentioned (if any)\n",
    "    3. Sentiment (positive, negative, neutral)\n",
    "    4. Main issue or request\n",
    "    5. Priority (high, medium, low)\n",
    "    6. Category (feedback, complaint, inquiry, other)\n",
    "    \n",
    "    Guidelines:\n",
    "    - For sentiment: positive means praise or satisfaction, negative means complaint or dissatisfaction, neutral for inquiries or mixed sentiment\n",
    "    - For priority: high for urgent issues or angry customers, medium for standard requests, low for general feedback\n",
    "    - For category: use your judgment based on the content\n",
    "    \n",
    "    Email:\n",
    "    {email}\n",
    "    \n",
    "    Format as JSON with keys: customer_name, product, sentiment, main_issue, priority, category\n",
    "    \"\"\"\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test both prompts\n",
    "basic_chain = LLMChain(llm=llm, prompt=basic_prompt)\n",
    "enhanced_chain = LLMChain(llm=llm, prompt=enhanced_prompt)\n",
    "\n",
    "for i, email in enumerate(test_emails):\n",
    "    print(f\"\\nEmail {i+1}:\\n{'='*50}\")\n",
    "    \n",
    "    print(\"\\nBasic Prompt Result:\")\n",
    "    basic_result = basic_chain.run(email=email)\n",
    "    print(basic_result)\n",
    "    \n",
    "    print(\"\\nEnhanced Prompt Result:\")\n",
    "    enhanced_result = enhanced_chain.run(email=email)\n",
    "    print(enhanced_result)\n",
    "    \n",
    "    print('\\n' + '-'*50)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Function to Parse Email"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def parse_email_content(email_body):\n",
    "    \"\"\"Parse email content to extract structured information.\n",
    "    \n",
    "    Args:\n",
    "        email_body: Raw email text\n",
    "        \n",
    "    Returns:\n",
    "        Dictionary with extracted information\n",
    "    \"\"\"\n",
    "    # Use the enhanced prompt\n",
    "    chain = LLMChain(llm=llm, prompt=enhanced_prompt)\n",
    "    \n",
    "    # Run chain\n",
    "    result = chain.run(email=email_body)\n",
    "    \n",
    "    # Parse JSON result\n",
    "    try:\n",
    "        parsed_result = json.loads(result.strip())\n",
    "        return parsed_result\n",
    "    except json.JSONDecodeError:\n",
    "        # Fallback in case of parsing error\n",
    "        return {\n",
    "            'customer_name': 'Unknown',\n",
    "            'product': 'Unknown',\n",
    "            'sentiment': 'neutral',\n",
    "            'main_issue': result,\n",
    "            'priority': 'medium',\n",
    "            'category': 'other'\n",
    "        }"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test the function\n",
    "for i, email in enumerate(test_emails):\n",
    "    print(f\"\\nEmail {i+1}:\\n{'='*50}\")\n",
    "    result = parse_email_content(email)\n",
    "    print(json.dumps(result, indent=2))\n",
    "    print('\\n' + '-'*50)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}